In [2]:
import requests
import json
import pandas as pd

In [8]:
houses = pd.read_csv('houses.csv', index_col=0)
houses.head()

,type,year,address,comment,category,num1,status_color,status
0,Дом жилой,XIX век,"набережная VI Армии, 41",NaN,региональная,Объект культурного наследия:№ 3500068000,Black,Утрачен полностью или значительно разрушен
1,Дом жилой,первая половина XIX века,"набережная VI Армии, 51",жилой дом,региональная,Объект культурного наследия:№ 3500000840,DarkBlue,Отреставрирован со значительными нарушениями п...
2,Дом жилой (с дворовым пристроем),первая половина XIX века,"набережная VI Армии, 81",NaN,региональная,Объект культурного наследия:№ 3500070001,Black,Утрачен полностью или значительно разрушен
3,Дом жилой,третья четверть XIX века,"набережная VI Армии, 91",NaN,региональная,Объект культурного наследия:№ 3500000842,Black,Утрачен полностью или значительно разрушен
4,Дом жилой,третья четверть XIX века,"набережная VI Армии, 93",NaN,региональная,Объект культурного наследия:№ 3500000844,Black,Утрачен полностью или значительно разрушен


У нас есть адреса домов, но нет их координат. Получить их можно через геокодер Яндекса.

In [9]:
TOKEN = None # your token key

In [5]:
def get_geo_data(**kwargs):
    API_URL = "https://geocode-maps.yandex.ru/1.x/"
    params = {**kwargs}
    data = requests.get(API_URL, params).json()
    return data['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['Point']['pos'].split(), data['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['metaDataProperty']['GeocoderMetaData']['text']

Проверяем всё ли работает нормально

In [10]:
get_geo_data(format='json', geocode='Вологда,+Воровского+улица,+дом+34', apikey=TOKEN)

(['39.870817', '59.225663'], 'Россия, Вологда, улица Воровского, 34')

In [ ]:
lat = []
lon = []
addr = []
for i in houses['address']:
    points, street  = get_geo_data(format='json', geocode=i, apikey=TOKEN)
    lat.append(points[0])
    lon.append(points[1])
    addr.append(street)
    print(i, '\n', street)
    print('\n')

In [12]:
houses['lat'] = lat
houses['lon'] = lon
houses['yandex_addr'] = addr

Теперь в датасете есть полный адрес, а также широта и долгота каждого дома.

In [13]:
houses.head()

,type,year,address,comment,category,num1,status_color,status,lat,lon,yandex_addr
0,Дом жилой,XIX век,"набережная VI Армии, 41",NaN,региональная,Объект культурного наследия:№ 3500068000,Black,Утрачен полностью или значительно разрушен,39.894361,59.235377,"Россия, Вологда, набережная 6-й Армии, 41"
1,Дом жилой,первая половина XIX века,"набережная VI Армии, 51",жилой дом,региональная,Объект культурного наследия:№ 3500000840,DarkBlue,Отреставрирован со значительными нарушениями п...,39.891415,59.232863,"Россия, Вологда, набережная 6-й Армии, 51"
2,Дом жилой (с дворовым пристроем),первая половина XIX века,"набережная VI Армии, 81",NaN,региональная,Объект культурного наследия:№ 3500070001,Black,Утрачен полностью или значительно разрушен,39.878623,59.228623,"Россия, Вологда, набережная 6-й Армии, 81"
3,Дом жилой,третья четверть XIX века,"набережная VI Армии, 91",NaN,региональная,Объект культурного наследия:№ 3500000842,Black,Утрачен полностью или значительно разрушен,39.882405,59.226685,"Россия, Вологда, набережная 6-й Армии, 91"
4,Дом жилой,третья четверть XIX века,"набережная VI Армии, 93",NaN,региональная,Объект культурного наследия:№ 3500000844,Black,Утрачен полностью или значительно разрушен,39.883043,59.226538,"Россия, Вологда, набережная 6-й Армии, 93"


In [15]:
houses.to_csv('houses.csv')